# Notebook for calibration of camera intrinsics and extrinsics 

### Includes 

In [1]:
import sys
import numpy as np
import glob
import cv2
import os

## ZED corner calibration

### Definitions

In [31]:

images_zed = glob.glob('../Images/zed*.jpg')

params = cv2.SimpleBlobDetector_Params()


params.filterByArea = True
params.minArea = 200
params.maxArea = 2250
print(params.minArea)
print(params.maxArea)

# Filter by Circularity (optional, based on grid shape).
params.filterByCircularity = False

# Filter by Convexity (optional).
params.filterByConvexity = True
params.minConvexity = 0.75

# Filter by Inertia (optional).
params.filterByInertia = False



detector = cv2.SimpleBlobDetector_create(params)

roi_size = 20


criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
kernel = np.ones((3,3),np.uint8)
search_window_size = (5,5)
zero_zone = (-1,-1)
grid_size = (6,10)

200.0
2250.0


### Corner detection

In [34]:
for fname in images_zed: 
    img = cv2.imread(fname)
    #cv2.imshow("temp",img)
    all_corners = []

    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 25, 255, cv2.THRESH_BINARY)
    
    thresh = cv2.bitwise_not(thresh)

    #thresh = cv2.morphologyEx(thresh,cv2.MORPH_OPEN, kernel)
    #thresh = cv2.morphologyEx(thresh,cv2.MORPH_TOPHAT, kernel)
    thresh = cv2.morphologyEx(thresh,cv2.MORPH_CLOSE,kernel)
    keypoints = detector.detect(thresh)
    
    im_with_keypoints = cv2.drawKeypoints(img, 
                                          keypoints, 
                                          np.array([]), 
                                          (0, 0, 255), 
                                          cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    points = np.array([kp.pt for kp in keypoints], dtype=np.float32)
    thresh_3_chnl = cv2.cvtColor(thresh,cv2.COLOR_GRAY2BGR)
    comparison_img = cv2.vconcat([thresh_3_chnl,im_with_keypoints])
    cv2.imshow("Thresholded and Keypoints", comparison_img)
   
    for point in points:
     x, y = int(point[0]),int(point[1])
     
     # Step 2 Define the ROI around center point
     x_start = max(0,x-roi_size)
     y_start = max(0,y-roi_size)

     x_end = min(gray.shape[1],x+roi_size)
     y_end = min(gray.shape[0],y+roi_size)

     roi = gray[y_start:y_end,x_start:x_end]

     corners = cv2.goodFeaturesToTrack(roi, maxCorners=4,qualityLevel=0.01, minDistance=5)

     if corners is not None:
        for corner in corners:
           cx, cy = corner.ravel()
           all_corners.append([cx+x_start,cy+y_start])
    all_corners_np = np.array(all_corners, dtype=np.float32).reshape(-1,1,2)

    refined_corners = cv2.cornerSubPix(
       gray,
       all_corners_np,
       search_window_size,
       zero_zone,
       criteria
    )
    for corner in refined_corners:
       cv2.circle(img,(int(corner[0][0]),int(corner[0][1])),2,(255,255,0),-1)
    
    #cv2.imshow('Refined corners',img)


    cv2.waitKey(0)

cv2.destroyAllWindows()

In [16]:
for fname in images_zed:
    img = cv2.imread(fname)

    all_corners = []

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    _, thresh = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY)

    ret, corners = cv2.findChessboardCorners(thresh, grid_size, None)
    
    if ret: 
        corners = cv2.cornerSubPix(gray, corners, search_window_size, zero_zone, criteria)


        cv2.drawChessboardCorners(img, grid_size, corners, ret)

        cv2.imshow('Detected corners', img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("Corners not found.")    

Corners not found.
Corners not found.
Corners not found.
Corners not found.
Corners not found.
Corners not found.
Corners not found.
Corners not found.
Corners not found.
Corners not found.
Corners not found.
Corners not found.
Corners not found.
Corners not found.
Corners not found.
Corners not found.
Corners not found.
Corners not found.
Corners not found.
Corners not found.
Corners not found.
Corners not found.


### Calibration of Intrinsics 

In [ ]:
# *Put code here, WIP* 

## Flir corner calibration

### Definitions 

In [60]:
images_flir = glob.glob('../Images/FLIR*.jpg')
output_path = "../Calib_corners/"
params = cv2.SimpleBlobDetector_Params()
params.filterByArea = True#True
params.minArea = 500
params.maxArea = 1700

# Filter by Circularity (optional, based on grid shape).
params.filterByCircularity = False

# Filter by Convexity (optional).
params.filterByConvexity = True
params.minConvexity = 0.875

# Filter by Inertia (optional).
params.filterByInertia = False

detector = cv2.SimpleBlobDetector_create(params)

roi_size = 40


criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
search_window_size = (5,5)
zero_zone = (-1,-1)

kernel = np.ones((3,3),np.uint8)

### Corner detection

In [66]:
#img_cnt = 1
for fname in images_flir: 
    img = cv2.imread(fname)
    #cv2.imshow("temp",img)
    all_corners = []
      
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5,5),0)
    thresh = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,73,-1)
    #_, thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)
    thresh = cv2.bitwise_not(thresh)
    #thresh = cv2.morphologyEx(thresh,cv2.MORPH_CLOSE, kernel)
    keypoints = detector.detect(thresh)
    
    im_with_keypoints = cv2.drawKeypoints(img, 
                                          keypoints, 
                                          np.array([]), 
                                          (0, 0, 255), 
                                          cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    points = np.array([kp.pt for kp in keypoints], dtype=np.float32)
    thresh_3_chnl = cv2.cvtColor(thresh,cv2.COLOR_GRAY2BGR)
    comparison_img = cv2.vconcat([thresh_3_chnl,im_with_keypoints])
    cv2.imshow("Thresholded and Keypoints", comparison_img)
    #output_name = output_path+"FLIR_comparisson_"+str(img_cnt)+'.jpg'
    #cv2.imwrite(output_name,comparison_img)
    #img_cnt+=1
    #cv2.imshow("Grayscale", gray)  
    for point in points:
     x, y = int(point[0]),int(point[1])
     
     # Step 2 Define the ROI around center point
     x_start = max(0,x-roi_size)
     y_start = max(0,y-roi_size)

     x_end = min(gray.shape[1],x+roi_size)
     y_end = min(gray.shape[0],y+roi_size)

     roi = gray[y_start:y_end,x_start:x_end]

     corners = cv2.goodFeaturesToTrack(roi, maxCorners=4,qualityLevel=0.05,useHarrisDetector=False, minDistance=30)

     if corners is not None:
        for corner in corners:
           cx, cy = corner.ravel()
           all_corners.append([cx+x_start,cy+y_start])
    all_corners_np = np.array(all_corners, dtype=np.float32).reshape(-1,1,2)

    refined_corners = cv2.cornerSubPix(
       gray,
       all_corners_np,
       search_window_size,
       zero_zone,
       criteria
    )
    for corner in refined_corners:
       cv2.circle(img,(int(corner[0][0]),int(corner[0][1])),2,(255,255,0),-1)
    
    #cv2.imshow('Refined corners',img)


    cv2.waitKey(0)

cv2.destroyAllWindows()